## Import

In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime

import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

In [3]:
# 커밋 날짜 자동설정
date = datetime.now().strftime("%Y-%m-%d %H:%M")
date

'2023-01-14 13:01'

## Tag Setting

In [4]:
taglist = pd.read_csv('TagList.csv')
taglist.fillna(0, inplace=True)


# man = ['a girl']
# hairColor = ['white', 'brown', 'red', 'pink', 'orange', 'yellow', 'Blonde', 'light green', 'green', 'sky blue', 'blue', 'purple', 'black', 'gray', 'golden']
# eyeColor = ['white', 'brown', 'red', 'pink', 'orange', 'yellow', 'Blonde', 'light green', 'green', 'sky blue', 'blue', 'purple', 'black', 'gray', 'golden']
# background = ['whiteground']
# dress = ['school uniform']
# bodyType = ['midium breast']
# others = ['sweat', 'blush', 'glittering', 'smoking', 'robot']
# tagLists = [man, hairColor, eyeColor, background, dress, bodyType, others]

In [5]:
taglist

,man,hair color,eye color,background,dress,body type,others
0,a girl,white,white,white background,NaN,NaN,NaN
1,NaN,brown,brown,brown background,NaN,NaN,NaN
2,NaN,red,red,red background,NaN,NaN,NaN
3,NaN,pink,pink,pink background,NaN,NaN,NaN
4,NaN,orange,orange,orange background,NaN,NaN,NaN
5,NaN,yellow,yellow,yellow background,NaN,NaN,NaN
6,NaN,Blonde,light green,light green background,NaN,NaN,NaN
7,NaN,light green,green,green background,NaN,NaN,NaN
8,NaN,green,sky blue,sky blue background,NaN,NaN,NaN
9,NaN,sky blue,blue,blue background,NaN,NaN,NaN


일단은 6가지의 대표적인 특징을 선택함   
사람은 여자만 있는 걸로. Novel AI에 학습된 남자 데이터는 보통 게이(...) 그림이 많아서 학습이 어려움   
머리 색, 눈동자 색은 적당한 색깔로 설정한 다음, 결과를 보며 수정할 생각   
배경, 의상, 체형은 가짓수가 많아서 내가 직접 보기에 괜찮은 것만 추려서 넣음   
중첩 가능한 태그인 others도 마찬가지.

## Make Tag

In [8]:
tagLength = 10
def get_tag(tagLength=tagLength, tagLists=taglist) -> str:
    '''
    태그 모음에서 랜덤하게 태그를 뽑아 태그 조합을 생성하는 함수입니다. 
    tagLength는 태그의 개수입니다.
    tagLength에서 others를 제외한 범주의 개수를 뺀 만큼이 중첩 가능한 태그의 개수입니다.
    masterpiece, best quality는 무조건 포함됩니다.
    '''

    result = 'masterpiece, best quality, '
    # 중첩 불가능한 태그 뽑기
    for i in range(len(tagLists) - 1):
        result += random.choice(tagLists[i]) + ', '

    # 중첩 가능한 태그 뽑기
    used = []
    for i in range(tagLength - len(tagLists) + 1):
        while True:
            tag = random.choice(tagLists[-1])
            if tag not in used:
                used.append(tag)
                break
        result += tag + ', '
    result = result[:-2]
    
    return result

## Get Image

In [10]:
URL = 'http://127.0.0.1:7860'
driver = webdriver.Edge(executable_path='msedgedriver.exe')
driver.get(url=URL)

# Novel AI 이외의 모델을 사용하려면
# 모델을 선택한 뒤 다음 셀 실행

C:\Users\36dav\AppData\Local\Temp\ipykernel_21032\2506742529.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge(executable_path='msedgedriver.exe')


In [12]:
element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, '#my-element'))
)

buttonGen = driver.find_element(By.CSS_SELECTOR,
    'button#txt2img_generate.gr-button.gr-button-lg.gr-button-primary')


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="setting_sd_model_checkpoint"]/label/select/option[1]"}
  (Session info: MicrosoftEdge=108.0.1462.76)
Stacktrace:
Backtrace:
	Microsoft::Applications::Events::EventProperties::SetProperty [0x00007FF667DF91D2+14306]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF667D80992+842866]
	(No symbol) [0x00007FF667A31D90]
	(No symbol) [0x00007FF667A75928]
	(No symbol) [0x00007FF667A75C10]
	(No symbol) [0x00007FF667AB2507]
	(No symbol) [0x00007FF667A95D6F]
	(No symbol) [0x00007FF667A6836E]
	(No symbol) [0x00007FF667AAF70C]
	(No symbol) [0x00007FF667A95B03]
	(No symbol) [0x00007FF667A673B8]
	(No symbol) [0x00007FF667A6638E]
	(No symbol) [0x00007FF667A67AE4]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF667CCFE18+119032]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF667CBD4D6+42934]
	Microsoft::Applications:

In [13]:
driver.close()